# Proyecto de obtención de datos contables de Bancos Privados - Banco Central del Uruguay

## Objetivo 

El objetivo del proyecto de datos, es la obtención de información contable histórica de los Bancos Privados que operan en el mercado uruguayo. 

## Fuente de datos 

La fuente de datos de esta información es el Banco Central del Uruguay (BCU), quien es órgano regulador de estas instituciones financieras, a quien deben reportar la información contable mensualmente.  

## Desarrollo del proyecto

In [6]:
# Se importan las librerías que se utilizarán durante el proyecto: 

import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import re

### Scrapping de archivos en la página del BCU

El objetivo de esta sección, será la obtención de las URL's de los archivos de información financiera de los Bancos Privados ("grupo997").

El BCU presenta mensualmente un "Boletin Mensual" desde donde iremos a buscar los archivos que necesitamos. La URL de estos listados tiene el siguiente formato: "https://www.bcu.gub.uy/Servicios-Financieros-SSF/Boletin%20SSF/ **"AÑO"** / **"MES"** /indice.htm". 

In [7]:
# La página web del BCU tiene certificado SSL, por lo que será necesario deshabilitar la verificación:
requests.packages.urllib3.disable_warnings() 

#Para acceder a los archivos, debo ir generando los links de los Boletines Mensuales: 

years = range(2016, 2024) #Se cuenta con datos mensuales desde el 2016 hasta la actualidad

meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Setiembre", "Octubre", "Noviembre", "Diciembre"] #Parte de la URL es el mes este formato

url_base = "https://www.bcu.gub.uy/Servicios-Financieros-SSF/Boletin%20SSF/{}/{}/indice.htm" #La URL tendrá este formato base y recorrerá las dos listas creadas para ir buscando los archivos que requerimos.

url_set = set() #Genero una lista donde se irán guardando las URL que encuentre


Con las listas generadas, comienzo a iterar en búsqueda de archivos XLS que contengan el número del grupo de los Bancos Privados: **"977"**

In [8]:
for year in years:
    for mes in meses:
        
        # Primero obtengo todos los links de la página con el URL creado:
        
        url = url_base.format(year, mes) # Construimos la URL con el año y mes actual
        response = requests.get(url, verify=False) # Realizamos la solicitud HTTP sin verificar el certificado SSL
        soup = BeautifulSoup(response.content, "html.parser") # Leemos el html para identificar todos los vínculos
        links = soup.find_all('a', href=True) # Encontramos todos los enlaces en la página

        # Luego busco los enlaces que contienen el número 997 en su texto, sean "XLS" y no sean de una institución en particular, sino del grupo977.
        
        for link in links:
            full_url = urljoin(url, link['href'])
            if '997' in link.text and full_url.lower().endswith('.xls') and 'institucion' not in full_url.lower():
                url_set.add(full_url) # Agregamos la URL a la lista

Luego de obtenidas todas las URL de los archivos que me interesan, las plasmo en un dataframe:

In [9]:
# DataFrame con las URL encontradas
df = pd.DataFrame(url_set, columns=['URL'])
print(df)

                                                  URL
0   https://www.bcu.gub.uy/Servicios-Financieros-S...
1   https://www.bcu.gub.uy/Servicios-Financieros-S...
2   https://www.bcu.gub.uy/Servicios-Financieros-S...
3   https://www.bcu.gub.uy/Servicios-Financieros-S...
4   https://www.bcu.gub.uy/Servicios-Financieros-S...
..                                                ...
91  https://www.bcu.gub.uy/Servicios-Financieros-S...
92  https://www.bcu.gub.uy/Servicios-Financieros-S...
93  https://www.bcu.gub.uy/Servicios-Financieros-S...
94  https://www.bcu.gub.uy/Servicios-Financieros-S...
95  https://www.bcu.gub.uy/Servicios-Financieros-S...

[96 rows x 1 columns]


### Descarga de archivos XLS

El objetivo de esta sección será utilizar las URL anteriormente generadas, para descargar los archivos y guardarlos en la ruta que se especifique. 

**En caso de querer utilizar el código, modificar la "ruta_descargas" según su convencia**

In [ ]:
ruta_descargas = "XXXXX" # Defino la ruta de descaga. 

# Comienzo a hacer la cescarga de archivos

for url in df['URL']:
    nombre_archivo = os.path.basename(url)
    
    # Quiero que al bajarse los archivos, se identifiquen por el mes y año al que corresponden, por lo que, los extraigo  año de la URL:
    match = re.search(r'(\d{4})/(\w+)/', url)
    if match:
        year = match.group(1)
        month = match.group(2)
        nombre_archivo = f"{month}_{year}_{nombre_archivo}"

    ruta_archivo = os.path.join(ruta_descargas, nombre_archivo)

    # Descargo el contenido de la URL
    response = requests.get(url, verify=False)

    # Verifico si la descarga fue exitosa
    if response.status_code == 200:
        # En caso de que sea exitosa, guardo el contenido en un archivo
        with open(ruta_archivo, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
        print(f"Descarga exitosa de {nombre_archivo}.")
    else:
        print(f"No se pudo descargar el contenido de la URL: {url}")

Descarga exitosa de Diciembre_2019_grupo997.xls.
Descarga exitosa de Junio_2017_grupo997.xls.
Descarga exitosa de Diciembre_2017_grupo997.xls.
Descarga exitosa de Abril_2017_grupo997.xls.
Descarga exitosa de Junio_2023_grupo997.xls.
Descarga exitosa de Agosto_2022_grupo997.xls.
Descarga exitosa de Setiembre_2020_grupo997.xls.
Descarga exitosa de Noviembre_2023_grupo997.xls.
Descarga exitosa de Enero_2016_grupo997.xls.
Descarga exitosa de Octubre_2022_grupo997.xls.
Descarga exitosa de Mayo_2020_grupo997.xls.
Descarga exitosa de Marzo_2020_grupo997.xls.
Descarga exitosa de Abril_2022_grupo997.xls.
Descarga exitosa de Abril_2021_grupo997.xls.
Descarga exitosa de Noviembre_2017_grupo997.xls.
Descarga exitosa de Octubre_2023_grupo997.xls.
Descarga exitosa de Junio_2019_grupo997.xls.
Descarga exitosa de Noviembre_2016_grupo997.xls.
Descarga exitosa de Setiembre_2017_grupo997.xls.
Descarga exitosa de Julio_2019_grupo997.xls.
Descarga exitosa de Agosto_2019_grupo997.xls.
Descarga exitosa de Fe